In [1]:
%pip install datasets langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


# Dataset Loading

- [Naver 경제, IT 뉴스기사 요약 데이터셋](https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko)

In [3]:
from datasets import load_dataset
import pandas as pd

# Hugging Face 데이터셋 로드
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")
dataset


DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [5]:
trainset = dataset['train']
print(type(trainset))
trainset

<class 'datasets.arrow_dataset.Dataset'>


Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 22194
})

In [6]:
df_train = trainset.to_pandas()
df_train.shape

(22194, 7)

In [7]:
df_train.head()

,date,category,press,title,document,link,summary
0,2022-07-03 17:14:37,economy,YTN,추경호 중기 수출지원 총력 무역금융 40조 확대,앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 ...,https://n.news.naver.com/mnews/article/052/000...,"올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, ..."
1,2022-07-04 08:07:12,economy,아시아경제,해산물·주류 무제한 인터컨티넨탈 뷔페 여름 한정 페스타 연다,문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준...,https://n.news.naver.com/mnews/article/277/000...,인터엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 쿨 섬머 페스타...
2,2022-07-05 18:01:01,IT과학,동아사이언스,한국인 생활 변화시킨 의사 마지막까지 영향력 컸던 과학자 이호왕 고려대 명예교수 소천,한탄바이러스 발견 노벨상 유력 후보로 자주 거론 한국을 대표하는 의학자이자 미생물학...,https://n.news.naver.com/mnews/article/584/000...,이 이호왕 고려대 명예교수는 바이러스의 병원체와 진단법 백신까지 모두 개발한 한국을...
3,2022-07-01 08:51:12,economy,뉴시스,에디슨이노 이승훈 제이스페이스 대표 사내이사 선임,기사내용 요약 우주발사체 사업 본격화 서울 뉴시스 김경택 기자 에디슨이노가 우주발사...,https://n.news.naver.com/mnews/article/003/001...,에디슨이노는 1일 임시주주총회를 통해 사명을 이노시스 로 변경하고 이승훈 제이스페이...
4,2022-07-01 16:11:01,economy,머니투데이,SK바사 해외 사업 조직 개편… 글로벌 탑티어 기업으로 성장 박차,SK바이오사이언스가 글로벌 사업의 고도화를 위해 조직 개편을 단행했다. SK바이오사...,https://n.news.naver.com/mnews/article/008/000...,SK바이오사이언스가 글로벌 사업의 고도화를 위해 기존 해외사업개발실을 백신사업뿐만 ...


In [8]:
df = df_train.query("category=='economy'").reset_index(drop=True)
df.shape

(17088, 7)

# 데이터셋 만들기
- 뉴스기사 제목과 뉴스기사를 이용해 그 기사에 영향을 받는 주가종목을 추론하는 모델을 만든다.
- 데이터 구성
  - **입력**: 뉴스 기사 제목, 뉴스 기사
  - **출력**: 뉴스기사가 주식에 영향을 주는지 여부, 부정적인 영향을 받는 회사와 이유, 긍정적인 영향을 받는 회사와 이유, 뉴스기사 요약
- Label을 LLM을 이용해 생성한다.
  - LLM을 이용해 데이터셋을 만든 이후 그 결과를 눈으로 검토해야 한다.

## Dataset 생성 Chain 구성

In [10]:
from tqdm import tqdm
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
template = '''# Instruction
당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.
사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.

## 분석 기준
1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.
2. 영향을 준다면 다음 항목을 출력하세요.
   - `"is_stock_related": true`
   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**
   - 뉴스에 **부정적** 영향을 받는 **회사이름들**
   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**
     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.
     - `None`, 유추, 추정, 일반 논평 금지합니다.
   - 뉴스 요약 (3줄 이내)
3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.
   - `"is_stock_related": false`
   - 뉴스 요약 (3줄 이내)

# 입력 데이터(뉴스기사)

{input}


# 출력 지시사항 (Output Indicator)

- {format_instructions}

## 출력 조건:
- 뉴스에 영향을 받은 회사들은 **반드시 한국 증시에 상장된 종목** 이어야 합니다.
- 뉴스에 있는 내용만 출력결과에 포함시킵니다.
- 긍정/부정 종목은 실제 뉴스기사에 영향을 받는 회사들만 포함하세요.
- 모든 문자열은 큰따옴표(`"`)로 감쌉니다.
- 문자열 안에 따옴표가 필요하면 작은따옴표(`'`)를 사용합니다.
- 모든 키(Key)는 출력 지시사항에 명시된 property들과 정확히 일치해야 합니다.
- `"positive_reasons"` 및 `"negative_reasons"`의 값은 `None`이 될 수 없습니다.
- json format을 잘 지켜 응답데이터를 만듭니다. 배열이나 object의 마지막 항목 뒤에 `,` 를 붙이지 마세요.
- 오직 유효한 JSON 문자열(UTF-8, RFC8259 준수)만 출력합니다.
- 절대 다른 텍스트, 주석, 설명, 코드 블록 표기(```), 또는 따옴표 외의 문자열을 추가하면 안 됩니다.

## 출력 예시 (Examples)

### 뉴스가 특정 주식종목들에 **긍정적 영향이 주는 경우**:
{{'is_stock_related': true,
 'negative_reasons': [],
 'negative_stocks': [],
 'positive_reasons': [{{'세라젬': '루게릭병 환우 지원 캠페인 후원과 의료가전 지원 등 사회공헌활동을 통해 기업 이미지와 브랜드 가치가 긍정적으로 부각됨'}}],
 'positive_stocks': ['세라젬'],
 'summary': '세라젬이 루게릭병 환우를 위한 아이스버킷 챌린지 런 행사를 후원하며 의료가전과 건강기능식품 등을 지원했다. 캠페인은 루게릭병 환우 지원과 기부 문화 확산을 목표로 한다. 세라젬은 다양한 사회공헌활동을 지속하고 있다.'
}}

### 뉴스의 내용이 특정 주식종목들에 **부정적 영향이 주는 경우**:
{{
    "is_stock_related": true,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [
        "포스코",
        "현대제철"
    ],
    "negative_reasons": [
        {{"포스코": "정부가 수입규제국 조사에 적극 대응하고 비관세장벽 해소를 위해 민관 협력 강화 방침을 밝혀 철강 분야에서 수출 피해 최소화 기대"}},
        {{"현대제철": "철강·금속 품목에 대한 수입규제 대응 강화로 불합리한 무역제한 조치 개선 가능성이 높아져 수출 환경 개선 기대"}}
    ],
    "summary": "산업부는 수입 규제국의 조사에 대응하고 비관세장벽 해소를 위한 협의를 진행했다. 규제 대상 국가는 26개국, 건수는 199건에 달한다."
}}

### **뉴스기사가 주식 종목과 관련 없는 경우**:
{{
    "is_stock_related": false,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [],
    "negative_reasons": [],
    "summary": "정황근 농림축산식품부 장관이 단순가공식품 부가가치세 면제 시행 상황을 점검했다. 된장, 고추장 코너를 방문하며 현장을 살폈다."
}}'''

In [13]:
class SummarySchema(BaseModel):
    is_stock_related: bool = Field(description="한국 주식과 관련있는 뉴스인지 여부")
    positive_stocks: list[str] = Field(description="뉴스기사에 긍정적인 영향을 받는 회사들의 이름들.")
    positive_reason: list[dict[str,str]] = Field(description='뉴스내용 중 positive_stocks에 있는 각 회사들에 긍정적 영향을 주는 내용. {"회사이름":"긍정적인 이유"}')
    negative_stocks: list[str] = Field(description="뉴스기사에 부정적인 영향을 받는 회사들의 이름들.")
    negative_reason: list[dict[str,str]] = Field(description='뉴스내용 중 negative_stocks에 있는 각 회사들에 부정적 영향을 주는 내용. {"회사이름":"부정적인 이유"}')
    summary: str = Field(description="뉴스기사 요약")

parser = JsonOutputParser(pydantic_object=SummarySchema)

prompt_template = ChatPromptTemplate(
    messages=[
        ("user", template), 
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_name = "gpt-4.1"
chain = prompt_template | ChatOpenAI(model=model_name) | parser

### 개별 데이터로 Chain 테스트

In [19]:
df.shape

(17088, 7)

In [17]:
from pprint import pprint

news_idx = 120
sample_news = df['title'].loc[news_idx]+"\n"+df['document'].loc[news_idx]
print(sample_news)

대상그룹 헌혈 캠페인 전국민 동참 레드챌린지
서울 연합뉴스 신현우 기자 5일 오전 서울 종로구 대상그룹 본사에서 이 회사 임직원들이 혈액 수급 안정 유도와 헌혈 참여 독려를 위한 전국민 동참 레드챌린지 캠페인을 홍보하고 있다. 대상 그룹은 7월 한 달간 전 국민을 대상으로 이 캠페인을 전개한다.


In [18]:
# 4.1 결과
response = chain.invoke(input={"input":sample_news})

pprint(response)

{'is_stock_related': True,
 'negative_reason': [],
 'negative_stocks': [],
 'positive_reason': [{'대상': "대상그룹이 임직원들과 함께 전국민을 대상으로 헌혈 캠페인 '레드챌린지'를 전개하며 기업의 "
                            '사회적 책임 활동과 이미지 제고에 힘쓰고 있음'}],
 'positive_stocks': ['대상'],
 'summary': '대상그룹이 전국민 헌혈 참여를 독려하는 레드챌린지 캠페인을 7월 한 달간 진행한다. 임직원들이 본사에서 홍보 활동에 '
            '참여했다. 혈액 수급 안정 목적이다.'}


## 답(label) 만들기

1. 뉴스제목(title)과 뉴스기사(document)를 합쳐서 입력데이터를 만든다.
2. 2의  입력데이터를 LLM에 요청해서 답변을 받은 뒤 DataFrame에 추가한다.

In [ ]:
# 1. K개 샘플링

sample_nums = 100  
sample_df = df.sample(sample_nums).reset_index(drop=True)
sample_df.shape

(100, 7)

In [23]:
# 2. 뉴스제목(title)과 뉴스기사(document)를 합쳐서 프롬프트를 생성한다.

articles = sample_df['title']+"\n"+sample_df['document']
articles.shape, type(articles)

((100,), pandas.core.series.Series)

In [25]:
# 3. LLM에 label 생성 요청

label_list = articles.apply(lambda x : chain.invoke({'input':x}))

In [30]:
articles[0]

'“MZ 세대로 소비층 확대⋯소맥 탈피하고 다양한 입맛 중시”\n이코노미조선 Interview 조선비즈 주류대상 심사위원들이 얘기하는 뉴 트렌드 2년간 이어진 팬데믹 pandemic·감염병 대유행 사태는 ‘혼술 혼자 마시는 술 ’과 ‘홈술 집에서 마시는 술 ’ 문화를 만들어 냈다. 유흥 시장이 타격을 입으면서 자연스레 소주와 기성 맥주 판매가 줄었고 MZ 세대 밀레니얼 Z 세대·1981 2010년생 를 중심으로 다양한 취향에 따른 와인과 수제맥주 위스키 프리미엄 소주 논알코올 비알코올 맥주 등 수요가 늘었다. 그리고 유흥 시장 대신 편의점과 대형마트가 주요 주류 구매 창구로 급부상했다. 취향이 다양해지고 다양한 술 구매가 가능하게 접근성이 좋아진 것도 ‘다양한 술 시대’를 열었다. ‘이코노미조선’은 다양한 술 소비 문화를 관찰하고 앞으로의 술 산업을 내다보는 ‘술 酒 소비 개성시대’를 기획했다. 편집자주 그래픽 이은현 그야말로 ‘술의 개성시대’가 열렸다. 코로나19 팬데믹 pandemic·감염병 대유행 을 계기로 홈 파티 문화가 자리 잡고 소비자들은 일종의 놀이처럼 개성에 따라 술을 선택하고 마신다. 공급자 측면에선 다양해진 소비자들의 요구를 충족하기 위한 노력이 계속되고 있다. 고급술로 인식되던 와인은 중저가 수입 와인이 늘어나면서 소비자층 확대를 노렸고 ‘아저씨의 술’로 인식되던 전통주는 MZ 세대 밀레니얼 Z 세대·1981 2010년생 를 중심으로 소셜미디어 SNS 에 사진을 찍어 올리기 좋은 ‘젊은 술’로 바뀌고 있다. 위스키 부문에서는 자신의 취향에 맞게 직접 술을 섞고 만들어 즐기는 ‘믹솔로지 Mixology ’ 트렌드와 집에서 직접 술을 제조해 마시는 홈텐딩 홈 바텐딩 현상도 눈에 띈다. 맥주는 쓴맛을 싫어하는 소비자들을 겨냥해 IPA 맥주 과일 맥주 논알코올 맥주 등 신제품이 쏟아지고 있고 소량 생산하는 수제맥주 크래프트 맥주 는 여전히 인기를 끌고 있다. 술의 소비 방식이 이전보다 다양해진 현실을 반영하듯 조선비즈 주류대상 출품 주종도 코로나1

In [27]:
label_list[0]

{'is_stock_related': True,
 'positive_stocks': ['오비맥주', '하이트진로'],
 'positive_reason': [{'오비맥주': "'파타고니아 바이세'가 주류대상에서 수상하며 소비자 호평과 브랜드 가치를 인정받음"},
  {'하이트진로': "프리미엄 수입 맥주 '싱하'가 수상작으로 선정되며 브랜드 가치와 수입주류 수요 확대에 긍정적 영향"}],
 'negative_stocks': [],
 'negative_reason': [],
 'summary': 'MZ세대의 입맛 변화로 와인, 수제맥주, 프리미엄 소주 등 다양한 주류 소비가 확대되고 있다. 오비맥주와 하이트진로 등 국내 주요 주류기업의 수상 제품에 대한 소비자 관심이 높아지고 있다. 홈술·혼술, 개성 음주 트렌드 확산이 주류시장을 변화시키고 있다.'}

In [31]:
# Label 타입 변환:  Dictionary 를 str로 변환.

label_list_str = label_list.apply(lambda x : str(x))
type(label_list_str[0]), label_list_str[0]

(str,
 '{\'is_stock_related\': True, \'positive_stocks\': [\'오비맥주\', \'하이트진로\'], \'positive_reason\': [{\'오비맥주\': "\'파타고니아 바이세\'가 주류대상에서 수상하며 소비자 호평과 브랜드 가치를 인정받음"}, {\'하이트진로\': "프리미엄 수입 맥주 \'싱하\'가 수상작으로 선정되며 브랜드 가치와 수입주류 수요 확대에 긍정적 영향"}], \'negative_stocks\': [], \'negative_reason\': [], \'summary\': \'MZ세대의 입맛 변화로 와인, 수제맥주, 프리미엄 소주 등 다양한 주류 소비가 확대되고 있다. 오비맥주와 하이트진로 등 국내 주요 주류기업의 수상 제품에 대한 소비자 관심이 높아지고 있다. 홈술·혼술, 개성 음주 트렌드 확산이 주류시장을 변화시키고 있다.\'}')

In [32]:
# Label을 데이터 프레임에 추가.
sample_df['label'] = label_list_str
sample_df.head()

,date,category,press,title,document,link,summary,label
0,2022-07-01 06:11:01,economy,조선비즈,“MZ 세대로 소비층 확대⋯소맥 탈피하고 다양한 입맛 중시”,이코노미조선 Interview 조선비즈 주류대상 심사위원들이 얘기하는 뉴 트렌드 2...,https://n.news.naver.com/mnews/article/366/000...,유흥 시장 대신 편의점과 대형마트가 주요 주류 구매 창구로 급부상하며 취향이 다양해...,"{'is_stock_related': True, 'positive_stocks': ..."
1,2022-07-01 10:42:13,economy,연합뉴스,유류세 인하폭 30%에서 37%로 확대,서울 연합뉴스 강민지 기자 정부 유류세 추가 인하분이 반영된 1일 오전 서울 강서구...,https://n.news.naver.com/mnews/article/001/001...,1일 유류세 추가 인하분이 반영된 1일 오전 서울 강서구 알뜰목화주유소에서 박일준 ...,"{'is_stock_related': False, 'positive_stocks':..."
2,2022-07-04 16:31:01,economy,데일리안,코빗 “50대 투자자가 가장 많아…증가율은 MZ가 탑”,40 50대 거래량 비중 56.8% MZ 세대 거래량 2.1%p 증가 코빗 상반기 ...,https://n.news.naver.com/mnews/article/119/000...,코인코빗의 투자자 중 올해 상반기 40 50대 중장년층의 거래량 비중은 56.8%로...,"{'is_stock_related': False, 'positive_stocks':..."
3,2022-07-01 14:36:16,economy,연합뉴스,여행소식 모리셔스 백신접종 시 검사 없이 입국,서울 연합뉴스 성연재 기자 백신을 접종한 사람은 모리셔스 입국 시 코로나19 신속항...,https://n.news.naver.com/mnews/article/001/001...,1일 모리셔스관광청은 백신을 접종한 사람은 모리셔스 입국 시 코로나19 신속항원검사...,"{'is_stock_related': False, 'positive_stocks':..."
4,2022-07-04 08:31:01,economy,매일경제,축제와 문화·서비스가 어우러진 청주 첫 복합문화공간 청주 센트럴시티,청주 센트럴시티 페스티벌가든 모습 사진 현대엔지니어링 최근 현대백화점이 전략적으로 ...,https://n.news.naver.com/mnews/article/009/000...,백화점 아울렛 등 쇼핑시설의 변신이 오프라인 만의 볼거리와 즐거움을 선사해 쇼핑 목...,"{'is_stock_related': True, 'positive_stocks': ..."


In [33]:
############ pickle로 저장
import os
os.makedirs("dataset", exist_ok=True)

output_file = "dataset/sample_df.pkl"

sample_df.to_pickle(
    output_file
)

# import pandas as pd
# sample_df = pd.read_pickle(output_file)

# 허깅페이스에 업로드

In [34]:
from huggingface_hub import login
from datasets import load_dataset

from dotenv import load_dotenv

load_dotenv()

True

In [35]:
import os
login(os.getenv("HUGGINGFACE_API_KEY"))

In [36]:
from datasets import Dataset

# Dataset으로 변환. 
dataset = Dataset.from_pandas(sample_df) # DataFrame을 Dataset으로 변환.
dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
    num_rows: 100
})

In [40]:
# dataset['label'][0]

In [44]:
# train/valid/test set으로 분리
dataset_dict = dataset.train_test_split(test_size=0.1)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 90
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 10
    })
})

In [45]:
# 데이터셋을 Huggingface hub 에 업로드.
dataset_id = "naver_economy_news_stock_instruct_dataset-100"
dataset_dict.push_to_hub(dataset_id)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kgmyh/naver_economy_news_stock_instruct_dataset-100/commit/324630cf1bb37819ef7efec3762f93b4e4f2ed55', commit_message='Upload dataset', commit_description='', oid='324630cf1bb37819ef7efec3762f93b4e4f2ed55', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kgmyh/naver_economy_news_stock_instruct_dataset-100', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kgmyh/naver_economy_news_stock_instruct_dataset-100'), pr_revision=None, pr_num=None)

In [46]:
# Dataset load

load_data = load_dataset("kgmyh/naver_economy_news_stock_instruct_dataset-100")

README.md:   0%|          | 0.00/615 [00:00<?, ?B/s]

c:\Users\Playdata\miniconda3\envs\lang_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--kgmyh--naver_economy_news_stock_instruct_dataset-100. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/203k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/40.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
load_data

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 90
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 10
    })
})

In [1]:
from datasets import load_dataset
d_id = "kgmyh/naver_economy_news_stock_instruct_dataset"
dataset = load_dataset(d_id)

In [49]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 1350
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 150
    })
})

In [2]:
trainset = dataset['train']

In [6]:
trainset[0]

{'date': '2022-07-05 13:41:15',
 'category': 'economy',
 'press': '뉴스1 ',
 'title': '대전도시공사 한국서비스 품질 우수기업 인증 획득',
 'document': '대전도시공사 김홍준 경영부사장 오른쪽 이 한국서비스진흥협회 관계자로부터 인증서를 받은 뒤 기념촬영을 하고 있다. 대전도시공사 제공 © 뉴스1 대전 뉴스1 김경훈 기자 대전도시공사는 산업통상자원부에서 주최하고 한국서비스진흥협회에서 주관하는 한국서비스 품질 우수기업 SQ 인증을 획득했다고 5일 밝혔다. SQ는 산업통상자원부 국가기술표준원의 평가지표를 기준으로 Δ고객접점 서비스 운영관리 Δ서비스 경영 성과 등 총 7개 항목 25개 지표에 대한 서류 심사 현장 평가 암행 평가 등 객관적 심사를 거쳐 인증서를 수여하고 널리 공표하는 제도다. 공사는 각종 사업을 추진하면서 적극적으로 시민의 목소리를 반영하고 입주민이 스스로 결정하는 주택 관리 착한 임대인 사업 29년 연속 흑자 경영과 이익배당 사회공헌 프로그램 등에서 높은 점수를 받았다. 공사 관계자는 “한국서비스품질 우수기업 인증 위상에 걸맞게 보다 품질 높은 서비스를 시민에게 제공하겠다”고 말했다.',
 'link': 'https://n.news.naver.com/mnews/article/421/0006198155?sid=101',
 'summary': '5일 5일 대전도시공사는 산업통상자원부에서 주최하고 한국서비스진흥협회에서 주관하는 총 7개 항목 25개 지표에 대한 서류 심사 현장 평가 암행 평가 등 객관적 심사를 거쳐 인증서를 수여하고 널리 공표하는 제도인 한국서비스 품질 우수기업 SQ 인증을 획득했다고 밝혔다.',
 'label': "{'is_stock_related': False, 'positive_stocks': [], 'positive_reasons': [], 'negative_stocks': [], 'negative_reasons': [], 'summary': '대전도시공사가 